In [1]:
# 간략화

#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import time

# 크롤링할 url 생성하는 함수 만들기(검색어, 시작 날짜, 종료 날짜, 최대 페이지)
def makeUrl(search, s_date, e_date, maxpage):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    url = [f"https://search.naver.com/search.naver?where=news&query={search}&sort=0&ds={s_date}&de={e_date}&nso=so%3Ar%2Cp%3Afrom{s_from}to{e_to}%2Ca%3A&start={i}" for i in range(1, int(maxpage) * 10, 10)]
    return url  

# html에서 원하는 속성 추출하는 함수 만들기(기사, 추출하려는 속성값)
def news_attrs_crawler(articles, attrs):
    attrs_content = [i.attrs[attrs] for i in articles]
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html 생성해서 기사 크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스 크롤링 시작#####
search = input("검색어 입력: ")  
s_date = input("시작 날짜 입력(예시: 2019.01.04): ")  #2019.01.04
e_date = input("종료 날짜 입력(예시: 2019.01.05): ")   #2019.01.05
maxpage = input("최대 크롤링할 페이지 수를 입력하세요: ")

# naver url 생성
url = makeUrl(search, s_date, e_date, maxpage)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
news_names = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    time.sleep(2)

    # 언론사명 가져 오기
    names = news_html.select('#contents > div.copyright > div > p')
    if names:
      name = names[0].string[12:-38]
    elif names != names:
      time.sleep(2)
      names = news_html.select('#content > div.end_ct > div > div.copyright > div > p')
      name = names[0].string[12:-38]
    else:
      name = ''

    # 뉴스 제목 가져 오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져 오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져 오기:  list합치기
    content = ''.join(str(content))

    # html 태그 제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>' # 태그 제거
    pattern2 = r'[\n\t<>]|&lt;|&gt;' # 특수 기호 제거

    title = re.sub(pattern=pattern1, repl='', string=str(title))
    title = re.sub(pattern=pattern2, repl='', string=str(title))

    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern3 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern3, '')
    content = re.sub(pattern2, '', content)[1:-1] # 기사 본문 맨 앞, 맨 뒤 [ ] 제거

    news_names.append(name)
    news_titles.append(title)
    news_contents.append(content)

# 날짜 가져 오기
    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time'][:10]
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='', string=str(news_date))

    # news_dates.append(news_date)
    news_dates.append(news_date[:10].replace('.', '-'))


###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates, 'name': news_names, 'title':news_titles, 'content':news_contents, 'link': final_urls})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)

#데이터 프레임을 엑셀 파일로 저장
outputFileName = f'{s_date} ~ {e_date}.xlsx'
news_df.to_excel(outputFileName, sheet_name='sheet1')

KeyboardInterrupt: ignored

In [ ]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     1311 non-null   object
 1   name     1311 non-null   object
 2   title    1311 non-null   object
 3   content  1311 non-null   object
 4   link     1311 non-null   object
dtypes: object(5)
memory usage: 51.3+ KB


In [ ]:
news_df[news_df['name'] == '']

,date,name,title,content,link
612,2021-12-14,,20억 아파트도 못 피했네…김새론 층간 소음 고백,"김새론, 성동구 위치한 아파트 거주매매가 최저 17억 원…최대 23억""같은 건물 사...",https://n.news.naver.com/mnews/article/015/000...
997,2021-11-17,,"김의철 KBS 사장 후보, 위장전입해 아파트 분양…""깊이 반성""",4억짜리 아파트 '다운계약서'…취등록세 1천400만원 적게 내 김의철 KBS 사장 ...,https://n.news.naver.com/mnews/article/001/001...
1122,None,,None,,https://sports.news.naver.com/news.nhn?oid=529...


In [ ]:
news_df[news_df['name'] == ''].count()

date       4
name       4
title      4
content    4
link       4
dtype: int64

In [ ]:
news_df[news_df['name'] == '']['link'].values

array(['https://n.news.naver.com/mnews/article/015/0004640235?sid=106',
       'https://n.news.naver.com/mnews/article/001/0012800282?sid=106',
       'https://sports.news.naver.com/news.nhn?oid=529&aid=0000061491',
       'https://n.news.naver.com/mnews/article/422/0000509250?sid=101'],
      dtype=object)

In [ ]:
news_df[news_df['content'] == '']

,date,name,title,content,link
1122,None,,None,,https://sports.news.naver.com/news.nhn?oid=529...


In [ ]:
news_df[news_df['content'] == '']['link'].values

array(['https://sports.news.naver.com/news.nhn?oid=529&aid=0000061491',
       'https://n.news.naver.com/mnews/article/422/0000509250?sid=101'],
      dtype=object)

In [ ]:
news_df.loc[1281, :]

date                                              2021-02-03
name                                                    이데일리
title                     “아파트값에 놀라 빌라로”…작년 12월 빌라매매 거래 ‘폭증’
content                                                     
link       https://n.news.naver.com/mnews/article/018/000...
Name: 1281, dtype: object

In [ ]:
news_df.loc[598	, 'content'] = '서울, 전세·매매지수 동반 하락…전국 아파트 ‘팔자’ 많아'

In [ ]:
news_df.loc[598	, 'content']

'서울, 전세·매매지수 동반 하락…전국 아파트 ‘팔자’ 많아'

In [ ]:
# ‘폭증’

In [ ]:
news_df.loc[1280, 'content'] = "서울의 아파트 매수 심리가 5주 연속 꺾인 것으로 나타났습니다. 한국부동산원에 따르면 이번 주 서울의 아파트 매매수급 지수는 101.9로, 지난 주보다 0.9포인트 떨어져 5주 연속 하락세를 보였습니다. 매매수급 지수는 수요와 공급 비중을 지수화한 것으로 기준선인 '100'을 넘어 높아질수록 매수 심리가 강함을 뜻합니다. 서울과 달리, 경기와 인천은 추석 연휴를 지나며 지수가 소폭 상승했습니다. 부동산원은 집값 급등에 따른 피로감과 대출 한도 축소 등이 겹친 결과로 분석했습니다."

In [ ]:
news_df.loc[1280, 'content']

"서울의 아파트 매수 심리가 5주 연속 꺾인 것으로 나타났습니다. 한국부동산원에 따르면 이번 주 서울의 아파트 매매수급 지수는 101.9로, 지난 주보다 0.9포인트 떨어져 5주 연속 하락세를 보였습니다. 매매수급 지수는 수요와 공급 비중을 지수화한 것으로 기준선인 '100'을 넘어 높아질수록 매수 심리가 강함을 뜻합니다. 서울과 달리, 경기와 인천은 추석 연휴를 지나며 지수가 소폭 상승했습니다. 부동산원은 집값 급등에 따른 피로감과 대출 한도 축소 등이 겹친 결과로 분석했습니다."

In [ ]:
news_df.loc[1123, 'content']

"     【서울=뉴시스】이영환 기자 = 21일 한국감정원의 '공동주택 실거래가격 지수' 자료에 따르면 4월 거래일 기준 서울 아파트값은 전월대비 0.38% 상승한 것으로 잠정 집계됐다. 이 지수는 실거래 가격만으로 가격 수준과 변동률을 나타낸 지표다. 원래 거래계약 신고가 모두 완료되는 시점(거래일로부터 60일) 이후 지수가 작성돼 2개월 이상 발표가 지체되지만 감정원은 적시성 보완을 위해 1개월 잠정지수를 생산 발표중이다. 지역별로 보면 도심권(종로·중·용산구)이 전달 대비 0.69% 하락했을 뿐 나머지 지역에서는 모두 상승 했다. 사진은 21일 오후 서울 송파구 롯데타워에서 바라본 아파트의 모습. 2019.06.21. 20hwan@newsis.com    [스포츠월드=정은희 온라인 뉴스 기자] 서울 용산구 한남동 고급주택 ‘한남더힐’이 올해 최고가 기록을 경신해 실거래가가 77억원을 갱신했다.   오늘(12일) 국토교통부 실거래가에 따르면 한남더힐 전용면적 243㎡는 지난달 4일 77억5000만원(1층)에 매매계약해 올해 전국에서 가장 비싸게 팔렸다. 이 단지의 올해 종전 최고가는 지난 4월과 9월 240㎡와 240㎡ 73억원이었다.   한남더힐은 지난 2015년부터 매년 최고 실거래가 1위를 기록하고 있다. 지난해 1월에는 244㎡가 84억원(3층)에 팔려 2006년 부동산 매매 실거래가 신고제 도입 이후 역대 최고가를 기록하기도 했다.   KB국민은행 부동산 리브온의 월간 주택가격동향에 따르면 지난달 서울 상위 20% 아파트의 평균 매매 시세는 19억1267만원으로 2008년 12월 관련 통계 공개 이후 처음으로 19억원을 넘었다.  사진=뉴시스 제공"

In [ ]:
news_df[news_df['name'] == '스포츠월드']['link'].values

array(['https://n.news.naver.com/mnews/article/396/0000562567?sid=106'],
      dtype=object)

In [ ]:
news_df[news_df['date'] == 'None']

,date,name,title,content,link
1122,None,,None,,https://sports.news.naver.com/news.nhn?oid=529...
1280,None,연합뉴스TV,None,,https://n.news.naver.com/mnews/article/422/000...


In [ ]:
news_df[news_df['title'] == '']

,date,name,title,content,link


In [ ]:
news_df[news_df['title'].isna()]

,date,name,title,content,link


In [ ]:
news_df.loc[1280, 'title']

'서울 아파트 매수심리 5주째 꺾여…경기·인천은 상승'

In [ ]:
news_df.loc[1280, 'title'] = '서울 아파트 매수심리 5주째 꺾여…경기·인천은 상승'

In [ ]:
news_df = news_df[news_df['name'].str.strip().astype(bool)]

In [ ]:
news_df[news_df['name'].isna()]

,date,name,title,content,link


In [ ]:
news_df[news_df['name'] == '']

,date,name,title,content,link


In [ ]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1308 entries, 0 to 1310
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     1308 non-null   object
 1   name     1308 non-null   object
 2   title    1308 non-null   object
 3   content  1308 non-null   object
 4   link     1308 non-null   object
dtypes: object(5)
memory usage: 61.3+ KB


In [ ]:
news_df.reset_index(drop = True, inplace = True)
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     1308 non-null   object
 1   name     1308 non-null   object
 2   title    1308 non-null   object
 3   content  1308 non-null   object
 4   link     1308 non-null   object
dtypes: object(5)
memory usage: 51.2+ KB


In [ ]:
news_df[news_df['title'].str.contains('홈즈')]

,date,name,title,content,link


In [ ]:
outputFileName = f'{s_date} ~ {e_date}.xlsx'
news_df.to_excel(outputFileName, sheet_name='sheet1')

In [ ]:
df_test = {'A': [1, 2, 1, 1],
           'B': [2, 3, 2, 2],
           'C': [3, 4, 3, 3],
           'D': [4, 5, 4, 2]}

In [ ]:
import pandas as pd
df_test = pd.DataFrame(df_test)
df_test

,A,B,C,D
0,1,2,3,4
1,2,3,4,5
2,1,2,3,4
3,1,2,3,2


In [ ]:
df_test.drop_duplicates(keep='first', ignore_index=True)

,A,B,C,D
0,1,2,3,4
1,2,3,4,5
2,1,2,3,2


In [ ]:
df_test

,A,B,C,D
0,1,2,3,4
1,2,3,4,5
2,1,2,3,4
3,1,2,3,2


In [2]:
# 간략화

#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import time

# 크롤링할 url 생성하는 함수 만들기(검색어, 시작 날짜, 종료 날짜, 최대 페이지)
def makeUrl(search, s_date, e_date, maxpage):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    url = [f"https://search.naver.com/search.naver?where=news&query={search}&sort=0&ds={s_date}&de={e_date}&nso=so%3Ar%2Cp%3Afrom{s_from}to{e_to}%2Ca%3A&start={i}" for i in range(1, int(maxpage) * 10, 10)]
    return url  

# html에서 원하는 속성 추출하는 함수 만들기(기사, 추출하려는 속성값)
def news_attrs_crawler(articles, attrs):
    attrs_content = [i.attrs[attrs] for i in articles]
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html 생성해서 기사 크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스 크롤링 시작#####
search = input("검색어 입력: ")  
s_date = input("시작 날짜 입력(예시: 2019.01.04): ")  #2019.01.04
e_date = input("종료 날짜 입력(예시: 2019.01.05): ")   #2019.01.05
maxpage = input("최대 크롤링할 페이지 수를 입력하세요: ")

# naver url 생성
url = makeUrl(search, s_date, e_date, maxpage)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
news_names = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    time.sleep(2)

    # # 언론사명 가져 오기
    # names = news_html.select('#contents > div.copyright > div > p')
    # if names:
    #   name = names[0].string[12:-38]
    # elif names != names:
    #   time.sleep(2)
    #   names = news_html.select('#content > div.end_ct > div > div.copyright > div > p')
    #   name = names[0].string[12:-38]
    # else:
    #   name = ''

    # 뉴스 제목 가져 오기
    # title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # if title == None:
    #     title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # # 뉴스 본문 가져 오기
    # content = news_html.select("div#dic_area")
    # if content == []:
    #     content = news_html.select("#articeBody")

    # # 기사 텍스트만 가져 오기:  list합치기
    # content = ''.join(str(content))

    # html 태그 제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>' # 태그 제거
    pattern2 = r'[\n\t<>]|&lt;|&gt;' # 특수 기호 제거

    # title = re.sub(pattern=pattern1, repl='', string=str(title))
    # title = re.sub(pattern=pattern2, repl='', string=str(title))

    # content = re.sub(pattern=pattern1, repl='', string=content)
    # pattern3 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    # content = content.replace(pattern3, '')
    # content = re.sub(pattern2, '', content)[1:-1] # 기사 본문 맨 앞, 맨 뒤 [ ] 제거

    # news_names.append(name)
    # news_titles.append(title)
    # news_contents.append(content)

# 날짜 가져 오기
    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time'][:10]
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='', string=str(news_date))

    # news_dates.append(news_date)
    news_dates.append(news_date[:10].replace('.', '-'))


# news_dates

###데이터 프레임으로 만들기###
import pandas as pd

# #데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates, 'link': final_urls})

# #중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)

# #데이터 프레임을 엑셀 파일로 저장
# outputFileName = f'{s_date} ~ {e_date}.xlsx'
# news_df.to_excel(outputFileName, sheet_name='sheet1')

검색어 입력: 태안 기름 유출
시작 날짜 입력(예시: 2019.01.04): 2007.12.30
종료 날짜 입력(예시: 2019.01.05): 2007.12.30
최대 크롤링할 페이지 수를 입력하세요: 10


100%|██████████| 53/53 [02:11<00:00,  2.48s/it]


In [3]:
news_df

,date,link
0,2007-12-30,https://n.news.naver.com/mnews/article/003/000...
1,2007-12-30,https://n.news.naver.com/mnews/article/214/000...
2,2007-12-30,https://n.news.naver.com/mnews/article/001/000...
3,None,https://sports.news.naver.com/news.nhn?oid=011...
4,2007-12-30,https://n.news.naver.com/mnews/article/015/000...
5,2007-12-30,https://n.news.naver.com/mnews/article/022/000...
6,2007-12-30,https://n.news.naver.com/mnews/article/032/000...
7,2007-12-30,https://n.news.naver.com/mnews/article/003/000...
8,2007-12-30,https://n.news.naver.com/mnews/article/047/000...
9,2007-12-30,https://n.news.naver.com/mnews/article/003/000...


In [4]:
news_df['link'].values

array(['https://n.news.naver.com/mnews/article/003/0000707426?sid=102',
       'https://n.news.naver.com/mnews/article/214/0000057320?sid=102',
       'https://n.news.naver.com/mnews/article/001/0001876571?sid=102',
       'https://sports.news.naver.com/news.nhn?oid=011&aid=0000213516',
       'https://n.news.naver.com/mnews/article/015/0001025386?sid=102',
       'https://n.news.naver.com/mnews/article/022/0000266092?sid=102',
       'https://n.news.naver.com/mnews/article/032/0000260070?sid=103',
       'https://n.news.naver.com/mnews/article/003/0000707173?sid=102',
       'https://n.news.naver.com/mnews/article/047/0000107469?sid=102',
       'https://n.news.naver.com/mnews/article/003/0000706672?sid=102',
       'https://n.news.naver.com/mnews/article/001/0001876757?sid=102',
       'https://n.news.naver.com/mnews/article/001/0001876516?sid=102',
       'https://n.news.naver.com/mnews/article/003/0000706673?sid=102',
       'https://n.news.naver.com/mnews/article/001/0001876816?si